# XNATpy

**XNATpy** is an XNAT client that exposes XNAT objects/functions as python objects/functions. The notebook provides a quick overview of using XNATpy. More information is availble in the [official documentation](https://xnat.readthedocs.io/en/latest/). 

## Setup

If XNATpy is not already installed

In [1]:
pip install xnat

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import xnat

## Connecting to an XNAT server

When connecting to an XNAT server you start by creating a connection. To avoid storing our credentials in our notebook we can use the XNAT environmental varibales.

In [3]:
connection = xnat.connect(os.environ['XNAT_HOST'], 
                          user=os.environ['XNAT_USER'], 
                          password=os.environ['XNAT_PASS'])

As of version 0.4.3, XNATpy supports the XNAT/Jupyter envrionment variables. So we can shorten this to

In [4]:
connection = xnat.connect()

## Exploring your XNAT server

When a session is established, it is fairly easy to explore the data on the XNAT server. The data structure of XNAT is mimicked as Python objects. The connection gives access to a listing of all projects, subjects, and experiments on the server.

In [5]:
connection.projects

<XNATListing {(C4KC-KiTS, C4KC-KiTS): <ProjectData C4KC-KiTS (C4KC-KiTS)>, (UPENN-GBM, UPENN-GBM): <ProjectData UPENN-GBM (UPENN-GBM)>, (MedNIST, MedNIST): <ProjectData MedNIST (MedNIST)>, (LIDC-IDRI, LIDC-IDRI): <ProjectData LIDC-IDRI (LIDC-IDRI)>}>

The XNATListing is a special type of mapping in which you can access elements by a primary key (usually the ID or Accession #) and a secondary key (e.g. the label for a subject or experiment). Selection can be performed the same as a Python dict.

In [6]:
project = connection.projects[0] # or connection.projects["C4KC-KiTS"]
project.subjects

<XNATListing {(XNAT_S02575, KiTS-00050): <SubjectData KiTS-00050 (XNAT_S02575)>, (XNAT_S02608, KiTS-00023): <SubjectData KiTS-00023 (XNAT_S02608)>, (XNAT_S02627, KiTS-00030): <SubjectData KiTS-00030 (XNAT_S02627)>, (XNAT_S02643, KiTS-00014): <SubjectData KiTS-00014 (XNAT_S02643)>, (XNAT_S02644, KiTS-00046): <SubjectData KiTS-00046 (XNAT_S02644)>, (XNAT_S02645, KiTS-00021): <SubjectData KiTS-00021 (XNAT_S02645)>, (XNAT_S02652, KiTS-00010): <SubjectData KiTS-00010 (XNAT_S02652)>, (XNAT_S02708, KiTS-00033): <SubjectData KiTS-00033 (XNAT_S02708)>, (XNAT_S02724, KiTS-00000): <SubjectData KiTS-00000 (XNAT_S02724)>, (XNAT_S02743, KiTS-00025): <SubjectData KiTS-00025 (XNAT_S02743)>, (XNAT_S02759, KiTS-00017): <SubjectData KiTS-00017 (XNAT_S02759)>, (XNAT_S02774, KiTS-00043): <SubjectData KiTS-00043 (XNAT_S02774)>, (XNAT_S02777, KiTS-00020): <SubjectData KiTS-00020 (XNAT_S02777)>, (XNAT_S02869, KiTS-00019): <SubjectData KiTS-00019 (XNAT_S02869)>, (XNAT_S02917, KiTS-00047): <SubjectData KiTS-000